#  1. Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('u.item.csv', sep= '|', header = None, encoding='latin-1')
movies.columns = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [3]:
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
ratings = pd.read_csv('u.data.csv', sep= '\t', header = None)
ratings.columns = ['user id', 'movie id', 'rating', 'timestamp']

In [5]:
ratings.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# 2. Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [6]:
ratings['user id'].value_counts().head(1)

405    737
Name: user id, dtype: int64

id пользователя, поставившего больше всего оценок = 405.Всего 737 оценок

# 3. Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

In [7]:
user405 = ratings[ratings['user id'] == 405]
user405.head()

,user id,movie id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


Далее нам уже не понадобится user id, там все равно единственное значение. timestamp нам тоже не понадобится, так как мы будем использовать не дату оценки, а дату выхода фильма

In [8]:
user405 = user405[['rating', 'movie id']]
user405.head()

,rating,movie id
12276,4,56
12383,1,592
12430,1,1582
12449,1,171
12460,1,580


## Для построения модели нам нужны признаки. В качестве таковых будем использовать:
Год выхода
Жанры
Общее количество оценок
Суммарную оценку

По этой причине я создала новый столбец с годом выхода фильма

# 4. Добавьте к датафрейму из задания 3 столбцы:
- По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
- столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей


In [9]:
import datetime

In [10]:
movies['release year'] = pd.to_datetime(movies['release date']).dt.strftime('%Y')

In [11]:
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,release year
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995


In [12]:
movies_genre = movies[['release year', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western','movie id']]

In [13]:
movies_genre.head()

,release year,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie id
0,1995,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1995,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,2
2,1995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
3,1995,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,4
4,1995,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,5


In [14]:
user405_genre = pd.merge(user405, movies_genre, how= 'left', on = 'movie id')
user405_genre.head()

,rating,movie id,release year,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,4,56,1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,592,1995,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,1582,1947,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,171,1991,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,1,580,1995,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [15]:
ratings['Count_score'] = ratings[['movie id', 'rating']].groupby(['movie id'])['rating'].transform('count')
ratings['Sum_score'] = ratings[['movie id', 'rating']].groupby(['movie id'])['rating'].transform('sum')
ratings.head()

,user id,movie id,rating,timestamp,Count_score,Sum_score
0,196,242,3,881250949,117,467
1,186,302,3,891717742,297,1236
2,22,377,1,878887116,13,28
3,244,51,2,880606923,81,280
4,166,346,1,886397596,126,459


In [16]:
ratings_for_merge = ratings[['Count_score', 'Sum_score', 'movie id']]
ratings_for_merge.head()

,Count_score,Sum_score,movie id
0,117,467,242
1,297,1236,302
2,13,28,377
3,81,280,51
4,126,459,346


In [17]:
user405_genre_score = pd.merge(user405_genre, ratings_for_merge, how = 'left', on = 'movie id').drop_duplicates()

In [18]:
user405_genre_score.head()

,rating,movie id,release year,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Count_score,Sum_score
0,4,56,1994,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,394,1600
394,1,592,1995,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,9,30
403,1,1582,1947,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
404,1,171,1991,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,65,252
469,1,580,1995,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,32,108


# 5. Сформируйте X_train, X_test, y_train, y_test

In [19]:
df = user405_genre_score.drop(['unknown'], axis=1)

In [20]:
df.head()

,rating,movie id,release year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Count_score,Sum_score
0,4,56,1994,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,394,1600
394,1,592,1995,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,9,30
403,1,1582,1947,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
404,1,171,1991,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,65,252
469,1,580,1995,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,32,108


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X = user405_genre_score.iloc[:, 2:].values
y = user405_genre_score.iloc[:, 0].values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 6. Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import  SVR

In [25]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1))
y_test = sc_y.fit_transform(y_test.reshape(-1,1))

In [26]:
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)

# 7. Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [27]:
mean_squared_error(y_test, y_pred)

0.6270489514415539

In [28]:
r2_score(y_test, y_pred)

0.37295104855844585

In [29]:
mean_absolute_error(y_test, y_pred)

0.5983824032769787

# 8. Загрузить данные в spark